In [7]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from drain3 import TemplateMiner #开源在线日志解析框架
from drain3.file_persistence import FilePersistence
from drain3.template_miner_config import TemplateMinerConfig
import time

In [8]:
data_train = pd.read_csv('./data/preliminary_sel_log_dataset.csv')
data_test = pd.read_csv('./data/preliminary_sel_log_dataset_a.csv')
data = pd.concat([data_train, data_test])

In [9]:
len(data)

493527

In [10]:
config = TemplateMinerConfig()
config.load('./drain3/drain3.ini') ## 这个文件在drain3的github仓库里有
config.profiling_enabled = False

In [11]:
drain_file = 'comp_a_sellog'
persistence = FilePersistence(drain_file + '.bin')
template_miner = TemplateMiner(persistence, config=config)

In [8]:
##模板提取
for msg in data.msg.tolist():
    template_miner.add_log_message(msg)
temp_count = len(template_miner.drain.clusters)

KeyboardInterrupt: 

In [7]:
for cluster in template_miner.drain.clusters:
    print(cluster)

ID=1     : size=8994      : System Boot Initiated <:*:> <:*:> <:*:> <:*:> <:*:> <:*:> | Asserted
ID=2     : size=11177     : Processor <:*:> <:*:> | Uncorrectable machine check exception | <:*:>
ID=3     : size=236287    : Memory <:*:> | <:*:> <:*:> | <:*:>
ID=4     : size=1285      : System <:*:> <:*:> <:*:> | <:*:> <:*:> | Asserted
ID=5     : size=12        : Cable/Interconnect #<:HEX:> | Config Error | <:*:>
ID=6     : size=4         : Add-in Card #<:HEX:> | Device Present
ID=7     : size=3787      : System <:*:> <:*:> <:*:> <:*:> | <:*:> <:*:> | Asserted
ID=8     : size=166       : Button #<:HEX:> | Power Button pressed | Asserted
ID=9     : size=247       : Power <:*:> #<:HEX:> | <:*:> <:*:> | <:*:>
ID=10    : size=5815      : Processor #<:HEX:> | <:*:> <:*:> | <:*:>
ID=11    : size=7247      : Drive Slot <:*:> | <:*:> <:*:> | <:*:>
ID=12    : size=8         : Power Unit #<:HEX:> | Device Present
ID=13    : size=36        : Processor #<:HEX:> | IERR | <:*:>
ID=14    : size=6      

In [8]:
## 筛选模板
# template_dic = {}
# size_list = []
# for cluster in template_miner.drain.clusters:
#     size_list.append(cluster.size)
# size_list = sorted(size_list, reverse=True)[:206] ## 筛选模板集合大小前200条，这里的筛选只是举最简单的例子。
# min_size = size_list[-1]

# for cluster in template_miner.drain.clusters: ## 把符合要求的模板存下来
#     print(cluster.cluster_id)
#     if cluster.size >= min_size:
#         template_dic[cluster.cluster_id] = cluster.size

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206


In [9]:
# temp_count_f = len(template_dic)

In [10]:
#将日志数据与筛选好的模板进行匹配
# def match_template(df, template_miner, template_dic):
#     msg = df.msg
#     cluster = template_miner.match(msg) # 匹配模板，由开源工具提供
#     if cluster and cluster.cluster_id in template_dic:
#         df['template_id'] = cluster.cluster_id # 模板id
#         df['template'] = cluster.get_template() # 具体模板
#     else:
#         df['template_id'] = 'None' # 没有匹配到模板的数据也会记录下来，之后也会用作一种特征。
#         df['template'] = 'None'
#     return df

In [11]:
# data = data.apply(match_template, template_miner=template_miner, template_dic=template_dic, axis=1)
# data.to_pickle('./drain3/' + drain_file +'_result_match_data.pkl') # 将匹配好的数据存下来
# print(len(data))
# print(data.head())

KeyboardInterrupt: 

In [12]:
df_data = pd.read_pickle('./drain3/' + drain_file + '_result_match_data.pkl')#读取匹配好模板的数据 
print(len(df_data))
print(df_data.head())

493527
             sn                 time  \
0  SERVER_25698  2020-10-09 08:32:21   
1  SERVER_25698  2020-10-09 07:43:48   
2  SERVER_25698  2020-10-09 08:16:22   
3  SERVER_25698  2020-10-09 05:46:41   
4  SERVER_25698  2020-10-09 12:59:13   

                                                 msg server_model  \
0   System Boot Initiated BIOS_Boot_Up | State As...          SM0   
1   System Boot Initiated BIOS_Boot_Up | State As...          SM0   
2   System Boot Initiated BIOS_Boot_Up | State As...          SM0   
3   System Boot Initiated BIOS_Boot_Up | State As...          SM0   
4   System Boot Initiated BIOS_Boot_Up | State As...          SM0   

   template_id                                           template  
0            1  System Boot Initiated <:*:> <:*:> <:*:> <:*:> ...  
1            1  System Boot Initiated <:*:> <:*:> <:*:> <:*:> ...  
2            1  System Boot Initiated <:*:> <:*:> <:*:> <:*:> ...  
3            1  System Boot Initiated <:*:> <:*:> <:*:> <:*:> ...

In [7]:
df_data[df_data['template_id']=='None'].head()

,sn,time,msg,server_model,template_id,template


#### 生成特征
此特征生成模块根据已匹配好的日志模板id数据，分别统计每个sn在m窗口内的每个模板id的数量，并构造n窗口内的滑窗统计量
在本示例中，m=1h，n=3h，滑窗统计量=‘sum’，这些参数均可根据需求按照自己的想法设置

In [13]:
def feature_generation(df_data, gap_list, model_name, log_source, win_list, func_list):

    dummy_list = set(df_data.template_id.unique())
    dummy_col = ['template_id_' + str(x) for x in dummy_list]

    for gap in gap_list:
        df_data['collect_time_gap'] = pd.to_datetime(df_data.collect_time).dt.ceil(gap)
        df_data = template_dummy(df_data)  #增加热编码列
        # print(df_data)

        df_data = df_data.reset_index(drop=True)
        df_data = df_data.groupby(['sn','collect_time_gap']).agg(sum).reset_index() #增加聚合列
        df_data = feature_win_fun(df_data, dummy_col, win_list, func_list, gap)
        df_data.to_pickle('./feature_data/template_fea/cpu_diag_comp_sel_log_all_feature'+gap+'.pkl') # 将构造好的特征数据存下来
        return df_data
    
def template_dummy(df):
    df_dummy = pd.get_dummies(df['template_id'], prefix='template_id')
    df = pd.concat([df[['sn','collect_time_gap']], df_dummy], axis=1)
    return df

def feature_win_fun(df, dummy_col, win_list, func_list, gap):
    drop_col = ['sn']
    merge_col = ['collect_time_gap']
    df_out = df[drop_col + merge_col]

    for win in win_list:
        for func in func_list:
            df_feature = df.groupby(drop_col).apply(rolling_funcs, win, func, dummy_col)
            df_feature = df_feature.reset_index(drop=True).rename(columns=dict(zip(dummy_col, map(lambda x: x + '_' +
                                                                  func + '_' + win, dummy_col))))
            df_out = pd.concat([df_out, df_feature], axis=1)
    return df_out

def rolling_funcs(df, window, func, fea_col):
    df = df.sort_values('collect_time_gap')
    df = df.set_index('collect_time_gap')
    df = df[fea_col]

    df2 = df.rolling(str(window) + 'h')

    if func == 'sum':
        df3 = df2.apply(sum_func)
    elif func == 'mean':
        df3 = df2.apply(mean_func)
    elif func == 'min':
        df3 = df2.apply(min_func)
    elif func == 'max':
        df3 = df2.apply(max_func)
    elif func == 'median':
        df3 = df2.apply(median_func)
    else:
        print('func not existed')
    return df3

def sum_func(series):
    return sum(series)
def mean_func(series):
    return np.mean(series)
def min_func(series):
    return min(series)
def max_func(series):
    return max(series)
def median_func(series):
    return np.median(series)

In [9]:
start = time.time()
df_data.rename(columns={'time':'collect_time'},inplace=True)
feature_generation(df_data, ['1h'],'', '', ['3'], ['sum','mean','max'])
print('耗时：{}秒'.format(time.time()-start))

耗时：2024.839473247528秒


In [14]:
df_data = pd.read_pickle(r'./feature_data/template_fea/cpu_diag_comp_sel_log_all_feature1h_3_sum.pkl') # 读取之前构造好的特征数据

In [15]:
df_data

,sn,collect_time_gap,template_id_1_sum_3,template_id_2_sum_3,template_id_3_sum_3,template_id_4_sum_3,template_id_5_sum_3,template_id_6_sum_3,template_id_7_sum_3,template_id_8_sum_3,...,template_id_197_sum_3,template_id_198_sum_3,template_id_199_sum_3,template_id_200_sum_3,template_id_201_sum_3,template_id_202_sum_3,template_id_203_sum_3,template_id_204_sum_3,template_id_205_sum_3,template_id_206_sum_3
0,000d33b21436,2020-09-02 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000d33b21436,2020-09-02 16:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,005c5a9218ba,2020-06-28 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0079283bde6e,2020-04-26 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,007bdf23b62f,2020-06-16 18:00:00,2.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35175,ffc229b6cd9a,2020-06-27 03:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35176,ffd44698a52b,2020-01-21 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35177,fff73a9e5bd5,2020-03-01 14:00:00,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35178,fffd22fffe19,2020-01-21 19:00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 加载embedding

In [ ]:
train_label["fault_time_label"] = train_label["fault_time"].dt.ceil(freq="1h")
train_label.reset_index(drop=True, inplace=True)
feat_cols = ["embedding_feat{}".format(i) for i in range(embedding_size)]


data = pd.DataFrame()
for i in tqdm(range(len(train_label))):
    sn_i = train_label.loc[i, "sn"]
    fault_time = train_label.loc[i, "fault_time_label"]
    start_time = fault_time - timedelta(days=1)
#     df_ = train_pre_sel_log[
#         (train_pre_sel_log["sn"] == sn_i) & (train_pre_sel_log["time_label"] < fault_time) & (train_pre_sel_log["time_label"] >= start_time)]
#     df_ = df_.groupby("sn")[feat_cols].mean().reset_index(drop=False)
    df_["fault_time_label"] = fault_time
    data = data.append(df_)
        
# 特征标签合并
logger.info(f"{len(data)}, {len(train_label)}")
data = pd.merge(data, train_label, on=["sn", "fault_time_label"], how="inner")
        
print(len(data))
print(data.head())

### 模型训练

In [16]:
df_train_label = pd.read_csv('./data/preliminary_train_label_dataset.csv')
df_train_label_s = pd.read_csv('./data/preliminary_train_label_dataset_s.csv')
df_train_label = pd.concat([df_train_label, df_train_label_s])  #12261+4410
df_train_label = df_train_label.drop_duplicates(['sn','fault_time','label'])

In [17]:
len(df_train_label)

16604

In [18]:
df_data_train = pd.merge(df_data[df_data.sn.isin(df_train_label.sn)],df_train_label, on='sn', how='left')
y = df_data_train['label']
x = df_data_train.drop(['sn','collect_time_gap','fault_time','label'],axis=1)

In [19]:
len(x)

44110

In [20]:
x.head()

,template_id_1_sum_3,template_id_2_sum_3,template_id_3_sum_3,template_id_4_sum_3,template_id_5_sum_3,template_id_6_sum_3,template_id_7_sum_3,template_id_8_sum_3,template_id_9_sum_3,template_id_10_sum_3,...,template_id_197_sum_3,template_id_198_sum_3,template_id_199_sum_3,template_id_200_sum_3,template_id_201_sum_3,template_id_202_sum_3,template_id_203_sum_3,template_id_204_sum_3,template_id_205_sum_3,template_id_206_sum_3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
clf = lgb.LGBMClassifier(max_depth=10,
        learning_rate=0.08, 
        n_estimators=10000)
clf.fit(x,y)

LGBMClassifier(learning_rate=0.08, max_depth=10, n_estimators=10000)

In [23]:
df_test_df = pd.read_csv('./data/preliminary_submit_dataset_a.csv',index_col=0).reset_index()
df_test = pd.merge(df_data[df_data.sn.isin(df_test_df.sn)], df_test_df, on='sn', how='left')

In [24]:
res = df_test[['sn','fault_time']]
x_test = df_test.drop(['sn','collect_time_gap','fault_time'],axis=1)

In [26]:
res['label'] = clf.predict(x_test)
res = res.sort_values(['sn','fault_time'])
res = res.drop_duplicates(['sn','fault_time'],keep='last')
res.to_csv('./submit/comp_a_result_2.csv', index=0)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices